## Below the steps to identify the most promising stocks using the EPS X appraoch

1. have an excel file with all the tickers extracted from a filter on tradingview
2. Compute the FP (see below) for all the companies
3. Compute PFV (see below) for the all the companies
4. Create a flag column "below eps_pfv" indicating whether the price is below pfv
5. take the tickers respecting the condition 4 and create history plots
6. select by hand the most promising stocks
7. for the selected ones get the price to pfv ratio
8. do an ascending sort of the stocks with the values computed in 7 and see the most promising tickers

### Formula

* $FP = EPS \cdot (1 + Growth)^{Years} \cdot PE$

* $PFV = \frac{FP}{(1 + Return)^{Years}}$

In the equations : 

* `FP` means *Future Price* and refers to the price that the stock should have in the future
* `PFV` mean *Present Fair Value* and refers to the price that the company should be trading today in the market
* `EPS` refers to the EPS that we see of the company today (now, avg TTM, MRQ, etc)
* `Growth` refers to what I think the company is going to grow in the following years
* `Years` are the number of years to take into consideration when doing the calculation
* `PE` refers to what I think is going to be the PE ratio after all the years considered. Thus, this is an estimated future PE ratio
* `Return` refers to what I think is going to be the avg yearly return of the investment during all the years considered

### Imports

In [1]:
import certifi
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from urllib.request import urlopen
from datetime import timedelta

### Functions

In [2]:
def compute_fp(
    eps:float,
    growth_value: float,
    years: int,
    future_pe: float,
) -> float:
    capped_growth_value = min(0.40, growth_value)
    capped_future_pe = min(20, future_pe)
    return eps * ((1 + capped_growth_value) ** years) * capped_future_pe

def compute_pfv(fp: float, return_value: float, years: int) -> float:
    capped_return_value = min(0.20, return_value)
    return fp / ((1 + capped_return_value) ** years)

def compute_pex_value(
    eps: float,
    growth_value: float,
    return_value: float,
    future_pe: float,
    years: int,
) -> float:
    fp = compute_fp(eps, growth_value, years, future_pe)
    pfv = compute_pfv(fp, return_value, years)
    return pfv

### Constants

In [3]:
KEY_PATH = "fmi-personal-key.txt"
with open(KEY_PATH, "r") as f:
    KEY = f.read()

In [4]:
with open("tickers_list_28062023.txt", "r") as f:
    TICKERS = f.read().split("\n")

In [5]:
TICKERS = list(set(TICKERS))

In [6]:
"CTHR" in TICKERS

True

Example of EPS X calculation for CTHR

There are 4 things that we need to decide to compute the value : 

* Return : I am going to use a standard of 20%
* Growth : I am going to use the data of the last 10 years and the what has been the avg growth. I am going to try it with EPS and then see if it's possible to do it with equity
* PE : The future PE ratio will be (10Y max - 10Ymin) / 2. I need to get the data from the last 10 years For the EPS it'll be straightforward, for the price I should match the eps of year X with the avg price of 3 months after the data of the report of year X
* Years : Number of years to consider in the calculations. I will use 7 (in between 5 and 10) cuz 5 is too short and 10 is too long

#### Return value

In [7]:
return_value = 0.2
return_value

0.2

#### Growth value

In [8]:
ticker = "CTHR"
url_income_stmt = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=10&apikey={KEY}"
response_income_stmt = urlopen(url_income_stmt, cafile=certifi.where())
data_income_stmt = response_income_stmt.read().decode("utf-8")
data_income_stmt = json.loads(data_income_stmt)

/var/folders/mr/pm_1h_jn1gs2n48rx1j1ygsr0000gn/T/ipykernel_9088/17637524.py:3: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response_income_stmt = urlopen(url_income_stmt, cafile=certifi.where())


In [9]:
all_eps = [element["eps"] for element in data_income_stmt]

In [10]:
all_eps

[0.08, 0.44, -0.22, 0.1, -0.0214, -0.22, -0.47, -0.65, -0.0649, 0.23]

In [11]:
all_growth = []
for idx in range(1, len(all_eps)):
    all_growth.append((all_eps[idx] - all_eps[idx-1]) / all_eps[idx-1])

In [12]:
all_growth

[4.5,
 -1.5,
 -1.4545454545454546,
 -1.214,
 9.280373831775702,
 1.1363636363636362,
 0.38297872340425543,
 -0.9001538461538462,
 -4.543913713405239]

In [13]:
# Cap the extreme growth rates for a more conservative approach
# Not the extrem drops since it'll help put numbers down
q1 = np.quantile(all_growth, 0.25)
q3 = np.quantile(all_growth, 0.75)
iqr = q3 - q1
lower_bound = q1 - 3*iqr
upper_bound = q3 + 3*iqr

all_growth_wo_extremes = [min(element, upper_bound) for element in all_growth]# if lower_bound < element < upper_bound]

0.5906466949726956

In [14]:
growth_value = np.mean(all_growth_wo_extremes)#np.mean(all_growth)

In [15]:
growth_value

0.5906466949726956

#### Future PE

In [16]:
# The reporting of a period is expected between 30 - 45 days after the ending of that period
all_prices = []
for single_income_stmt in data_income_stmt[::-1]:
    reporting_window_start = pd.to_datetime(single_income_stmt["date"]) + pd.DateOffset(days=30)
    reporting_window_start = str(reporting_window_start.date())
    reporting_window_end = pd.to_datetime(single_income_stmt["date"]) + pd.DateOffset(days=46)
    reporting_window_end = str(reporting_window_end.date())
    
    url_prices =  f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={reporting_window_start}&to={reporting_window_end}&apikey={KEY}"
    response_prices = urlopen(url_prices, cafile=certifi.where())
    data_prices = response_prices.read().decode("utf-8")
    data_prices = json.loads(data_prices)
    
    price_at_report = np.mean([element["low"] for element in data_prices["historical"]])
    
    all_prices.append(price_at_report)

/var/folders/mr/pm_1h_jn1gs2n48rx1j1ygsr0000gn/T/ipykernel_9088/3319864374.py:10: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response_prices = urlopen(url_prices, cafile=certifi.where())


In [17]:
all_prices

[3.7731538461538467,
 4.059833333333333,
 1.6518181818181816,
 0.8430099999999999,
 0.9940923076923075,
 1.3241153846153846,
 1.430392307692308,
 0.6971416666666667,
 2.601818181818182,
 1.3108909090909093]

In [18]:
# I don't like a pe of zero cuz we mask very big drops that could hide info
historical_pe = list(np.array(all_prices) / np.array(all_eps))
historical_pe = [val if val > 0 else 0 for val in historical_pe]
historical_pe

[47.164423076923086,
 9.226893939393937,
 0,
 8.4301,
 0,
 0,
 0,
 0,
 0,
 5.6995256916996055]

In [19]:
future_pe = np.mean(historical_pe)

In [20]:
future_pe

7.052094270801663

#### Years

In [21]:
years = 7

#### Calculations

Reminder:

* $FP = EPS \cdot (1 + Growth)^{Years} \cdot PE$

* $PFV = \frac{FP}{(1 + Return)^{Years}}$

In [27]:
eps = all_eps[0] # latest yearly EPS

fp = eps * ((1 + growth_value) ** years) * future_pe
pfv = fp / ((1 + return_value) ** years)

In [28]:
buy_price = pfv * 0.5
print(f"Present Faire Value:   {round(pfv, 2)}")
print(f"Buying Price:          {round(buy_price, 2)}")

Present Faire Value:   4.06
Buying Price:          2.03


**NOTES** : 

* The higher the future PE ratio the higher the present faire value. -> Choose a conservative PE ratio
* The higher the number of years the higher the present faire value (assuming the same number of years considered for both future price and present faire value). -> Choose a mid-term horizon
* For a fixed rate of return (return_value), the greater the growth_value the greater the present faire value. -> Choose a conservative growth value
* For a fixed growth_value, the greater the rate of return the lower the present faire value. -> Choose a slightly high rate of return

In [31]:
pfv_conservative = compute_pex_value(eps, growth_value, return_value, future_pe, years)

In [32]:
buy_price_conservative = pfv_conservative * 0.5
print(f"Present Faire Value:   {round(pfv_conservative, 2)}")
print(f"Buying Price:          {round(buy_price_conservative, 2)}")

Present Faire Value:   1.66
Buying Price:          0.83
